
#
# MODULO 07 - EJERCICIO 04-A
# ALEXIS YURI M.


Primero es necesario instalar y configurar PySpark en el notebook de Colab. Para eso se ejecutan las siguientes celdas para instalar las librerías necesarias y crear un contexto de Spark.




In [1]:
# Se instala PySpark y findspark
!pip install pyspark findspark

# Se inicializa findspark para encontrar la instalación de Spark.
import findspark
findspark.init()

# Se importa SparkContext.
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Se crea una sesión de Spark si no existe.
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder.master("local[*]").appName("AnalisisAgricola").getOrCreate()

sc = spark.sparkContext

print("SparkSession está lista para usar.")

SparkSession está lista para usar.


Paso 1: Se carga el archivo CSV, para eso se debe subir el archivo 1. datos_agricolas.csv en el panel de archivos.


In [2]:
# Se carga el archivo CSV en un DataFrame.
df_agricola = spark.read.csv('1. datos_agricolas.csv', header=True, sep=';', inferSchema=True , encoding='utf-8')


Paso 2: Se muestran las primera filas y se analiza la estructura del Dataframe.


In [3]:
# Se muestran las primeras filas, los tipos de datos y el esquema para confirmar que la carga fue exitosa.
print("Esquema del DataFrame:")
df_agricola.printSchema()
print("\nPrimeras 5 filas:")
df_agricola.show(5)
print("\nResumen del Dataframe:")
df_agricola.describe()


Esquema del DataFrame:
root
 |-- Year: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Cultivo: string (nullable = true)
 |-- Superficie_Ha: integer (nullable = true)
 |-- Produccion_Ton: integer (nullable = true)
 |-- Precio_Ton: integer (nullable = true)
 |-- Costos_Insumos: integer (nullable = true)
 |-- Ingresos: integer (nullable = true)
 |-- Utilidad: integer (nullable = true)


Primeras 5 filas:
+----+---+------+-------+-------------+--------------+----------+--------------+--------+--------+
|Year|Mes|Region|Cultivo|Superficie_Ha|Produccion_Ton|Precio_Ton|Costos_Insumos|Ingresos|Utilidad|
+----+---+------+-------+-------------+--------------+----------+--------------+--------+--------+
|2023|  7|   Sur|Manzana|           34|          4583|       378|          5267| 1730770| 1725503|
|2024|  5|Centro| Tomate|          253|          3501|       106|          6988|  372681|  365693|
|2023|  3|   Sur|   Soja|          430|   

DataFrame[summary: string, Year: string, Mes: string, Region: string, Cultivo: string, Superficie_Ha: string, Produccion_Ton: string, Precio_Ton: string, Costos_Insumos: string, Ingresos: string, Utilidad: string]

Paso 3: Se registra Dataframe como vista temporal.


In [4]:
# Se crea una vista temporal para poder usar Spark SQL.
df_agricola.createOrReplaceTempView("datos_agricolas")

print("Vista temporal registrada.")

Vista temporal registrada.



Paso 4: Se ejecutan consultas con Spark SQL.

In [5]:
# Consulta 1: Rendimiento promedio por región.
# El rendimiento promedio se calcula como Producción en Toneladas dividido por Superficie en Hectáreas.
# Se agrupa el resultado por región.


print("Rendimiento promedio por región:")

consulta1=spark.sql("""
    SELECT
        Region,
        AVG(Produccion_Ton / Superficie_Ha) AS Rendimiento_Promedio
    FROM
        datos_agricolas
    GROUP BY
        Region
""")
consulta1.show()


# Consulta 2: Top 3 cultivos más rentables.
# La rentabilidad se mide por la suma total de Utilidad.
# Se agrupa por cultivo y se muestran los 3 mayores en orden descendente.

print("Top 3 cultivos más rentables:")
consulta2=spark.sql("""
    SELECT
        Cultivo,
        SUM(Utilidad) AS Utilidad_Total
    FROM
        datos_agricolas
    GROUP BY
        Cultivo
    ORDER BY
        Utilidad_Total DESC
    LIMIT 3
""")
consulta2.show()


# Consulta 3: Ingresos totales para la Papa en la región Norte en el año 2023.
# Se suman todos los ingresos del año 2023 en la region Norte para el Cultivo de la Papa.

print("Ingresos totales para la Papa en la región Norte en 2023:")
consulta3=spark.sql("""
    SELECT
        SUM(Ingresos) AS Ingresos_Papa_Norte_2023
    FROM
        datos_agricolas
    WHERE
        Year = 2023 AND Region = "Norte" AND Cultivo = "Papa"
""")
consulta3.show()



# Parada ordenada de Spark.
spark.stop()
print("\n")
print("Spark detenido.")


Rendimiento promedio por región:
+------+--------------------+
|Region|Rendimiento_Promedio|
+------+--------------------+
|   Sur|   23.34617487434811|
| Norte|  27.802923541185088|
|Centro|  25.610502052637127|
+------+--------------------+

Top 3 cultivos más rentables:
+-------+--------------+
|Cultivo|Utilidad_Total|
+-------+--------------+
|Manzana|     206003764|
| Tomate|     195542018|
|    Uva|     193713685|
+-------+--------------+

Ingresos totales para la Papa en la región Norte en 2023:
+------------------------+
|Ingresos_Papa_Norte_2023|
+------------------------+
|                35488409|
+------------------------+



Spark detenido.
